In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, dataloader

In [ ]:
def sinDataLoader()
def return_sin_wave(self, sinmod, offset = np.pi*2,step = 1/1000):
    x = np.linspace(0,int(offset),1/step)
    y = np.sin(x*(sinmod))
    return y